In [20]:
import os
os.chdir('/home/guohao826/AppAlgorithmFace')
import cv2
import uuid
import base64
import time
import logging
import tempfile
import numpy as np
from src.config import Config
from src.service.alive_alg.alive import LiveFaceDetection, check_alive
from src.service.face_alg.detect_inference import DetectInference
from src.service.face_alg.landmark_inference import LandmarkInference
from src.service.face_alg.antispoofing_inference import AntiSpoofingInference

from src.service.face_alg.service_demo import PthExtractor, detect_align, get_sim
from src.service.image import ImageService
from src.util.decorator import catch
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

## 加载模型

In [21]:
detector = DetectInference(Config.DETECT_MODEL_PATH, Config.DETECT_GPUID, Config.DETECT_INPUT_SHAPE,
                               Config.DETECT_THRESHOLD)
landermark_detector = LandmarkInference(Config.DETECT_LANDMARK_MODEL_PATH, Config.DETECT_GPUID)
# 活体模型
# alive_model = LiveFaceDetection(Config.ALIVE_MODEL_PATH, Config.ALIVE_THRESHOLD_ORIENTATION,
#                                     Config.ALIVE_THRESHOLD_MOUSE, Config.ALIVE_THRESHOLD_MOVE,
#                                     Config.ALIVE_THRESHOLD_EYE, Config.ALIVE_THRESHOLD_ALIGH,
#                                     Config.ALIVE_WIN_WIDTH, Config.ALIVE_WIN_HEIGHT)

In [22]:
##预处理
def preprocess(frame):
    # img_w = frame.shape[1]
    # img_h = frame.shape[0]
    # if img_w > img_h:
    #     frame = np.rot90(frame)

    dim = frame.shape
    resize_w = alive_model.WIN_WIDTH
    resize_h = int(resize_w * dim[0] / dim[1])

    frame = cv2.resize(frame, (resize_w, resize_h))
    return frame

In [29]:
##检测人脸区域
def detect_face(frame):
    bboxes,lads = detector.detect([frame])
    bbox= bboxes[0]
    return bbox

def plot_one_box(x, im, label=None, line_thickness=3):
    tl = line_thickness or round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
    color = [0,0,255]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    c3 = c1[0],c2[1]
    cv2.rectangle(im, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    
def plot_face_region(bboxs,im):
    imm = im.copy()
    for bbox in bboxs:
        plot_one_box(bbox,imm)
    return imm
    
    
def plot_lads(landermarks,frame):
    res1 = frame.copy()
    for a in landermark_98.getA()  :
        cv2.circle(res1,(int(a[0]),int(a[1])),2,(0,0,255),-1)
    return res1

def get_mouse_status(landmarks_in):
    mouseArea = []
    for i in range(48, 60):
        point = landmarks_in[i]
        mouseArea.append([int(point[0, 0]), int(point[0, 1])])
    mouseArea = np.array(mouseArea)
    mouseSize = cv2.contourArea(mouseArea)
    mouseLength = landmarks_in[54][0, 0] - landmarks_in[48][0, 0]
    mouseRatio = mouseSize / (mouseLength * mouseLength)
    return mouseRatio

def get_eyes_status(landmarks_in):

    landmarks_eye = landmarks_in[60:68]
    landmarks_right = landmarks_in[68:76]

    left_err = alive_model.eye_aspect_ratio_eight_points(landmarks_eye)
    right_err = alive_model.eye_aspect_ratio_eight_points(landmarks_right)
    mean_err = (left_err + right_err) / 2.0
    return mean_err


def getfaceVar(image,face_box, img_w, img_h):
    left_x = int(face_box[0])
    left_y = int(face_box[1])
    right_x = int(face_box[2])
    right_y = int(face_box[3])

    if left_x < 1:
        left_x = 0
    if left_y < 1:
        left_y = 0
    if right_x > (img_w - 1):
        right_x = (img_w - 1)
    if right_y > (img_h - 1):
        right_y = (img_h - 1)

    crop_face = frame[left_y:right_y, left_x:right_x]
    # image = cv2.cvtColor(np.asarray(PIl_image),cv2.COLOR_RGB2BGR)
    img2gray = cv2.cvtColor(crop_face, cv2.COLOR_BGR2GRAY)
    imageVar = cv2.Laplacian(img2gray, cv2.CV_64F).var()
    return imageVar


def getImageVar(image):
    # image = cv2.cvtColor(np.asarray(PIl_image),cv2.COLOR_RGB2BGR)
    img2gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imageVar = cv2.Laplacian(img2gray, cv2.CV_64F).var()
    return imageVar


def isLightnessover( frame, face_box, img_w, img_h):
    left_x = int(face_box[0])
    left_y = int(face_box[1])
    right_x = int(face_box[2])
    right_y = int(face_box[3])

    if left_x < 1:
        left_x = 0
    if left_y < 1:
        left_y = 0
    if right_x > (img_w - 1):
        right_x = (img_w - 1)
    if right_y > (img_h - 1):
        right_y = (img_h - 1)

    crop_face = frame[left_y:right_y, left_x:right_x]
    img_hsv = cv2.cvtColor(crop_face, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img_hsv)
    bright_value = np.mean(np.mean(v))

    # value from [0-255] 0 is dark
    if bright_value > 230:
        return False
    return True

In [31]:
dirpath = '/home/guohao826/AppAlgorithmFace/process_file/imgs'
root = '/home/guohao826/AppAlgorithmFace/process_file'
filenames = os.listdir(dirpath)
os.makedirs(os.path.join(root,'no_face'),exist_ok=True)
os.makedirs(os.path.join(root,'mohu'),exist_ok=True)
os.makedirs(os.path.join(root,'light_lack'),exist_ok=True)
os.makedirs(os.path.join(root,'light_over'),exist_ok=True)
os.makedirs(os.path.join(root,'small_face'),exist_ok=True)
os.makedirs(os.path.join(root,'ola'),exist_ok=True)
os.makedirs(os.path.join(root,'eye_close'),exist_ok=True)
os.makedirs(os.path.join(root,'mouse_open'),exist_ok=True)

In [32]:
for filename in tqdm(filenames):
    try:
        #filename = 'base_20211219174036f5fd906f42ca40db82b532e07bf5215a.jpg'
        img_path = os.path.join(dirpath,filename)
        img = cv2.imread(img_path)
        #img = cv2.imread('./eye.jpg')
        frame = preprocess(img)
        bounding_boxes = detect_face(frame)
        dim = frame.shape
        img_h = dim[0]
        img_w = dim[1]
        if len(bounding_boxes) > 0:
            index = alive_model.maxbox(bounding_boxes)
            box = bounding_boxes[index]
            isLightover = isLightnessover(frame, box, img_w, img_h)
            isLightFace = alive_model.isLightnessEnough(frame, box, img_w, img_h)
            isBigFace = alive_model.isBig(box, img_w, img_h)
            ImageVar = getImageVar(frame)
            if ImageVar < 10:
                target_path = os.path.join(root,'mohu',filename)
                shutil.copyfile(img_path,target_path)
            if isLightFace == False:
                target_path = os.path.join(root,'light_lack',filename)
                shutil.copyfile(img_path,target_path)
            if isBigFace == False:
                target_path = os.path.join(root,'small_face',filename)
                shutil.copyfile(img_path,target_path)
            if isLightover == False:
                target_path = os.path.join(root,'light_over',filename)
                shutil.copyfile(img_path,target_path)
            ##关键点检测
            faceimg, face_size, (x1, y1) = landermark_detector.preprocess(box, frame)
            landmark_QC = landermark_detector.inference(faceimg, face_size)
            landermark_98 = alive_model.landermark_plus_leftcorner(landmark_QC, np.array([x1, y1]))
            landmarks = alive_model.index_98to68(landmark_QC, np.array([x1, y1]))
            reprojectdst, euler_angle = alive_model.get_head_pose(landmarks)
            aligned_sum = abs(euler_angle[0, 0]) + abs(euler_angle[1, 0]) + abs(euler_angle[2, 0])
            eye_status = get_eyes_status(landermark_98)
            mouse_status = get_mouse_status(landmarks)
            if aligned_sum > 80 :
                target_path = os.path.join(root,'ola',filename)
                shutil.copyfile(img_path,target_path)
            elif eye_status<0.11:
                target_path = os.path.join(root,'eye_close',filename)
                shutil.copyfile(img_path,target_path)
            elif mouse_status>0.6:
                target_path = os.path.join(root,'mouse_open',filename)
                shutil.copyfile(img_path,target_path)
        else:
            target_path = os.path.join(root,'no_face',filename)
            shutil.copyfile(img_path,target_path)
    except:
        print(filename)
            


  0%|          | 0/20 [00:00<?, ?it/s]

 55%|█████▌    | 11/20 [00:00<00:00, 48.00it/s]

8.png
17.png
15.png
14.png
11.png
5.png
12.png
9.png
6.png
3.png
16.png


100%|██████████| 20/20 [00:00<00:00, 46.80it/s]

13.png
18.png
4.png
2.png
30.png
21.png
7.png
1.png
10.png


In [26]:
plt.imshow(immm)

NameError: name 'immm' is not defined

In [ ]:
immm = plot_face_region(bounding_boxes,frame)

NameError: name 'bounding_boxes' is not defined

In [ ]:
plt.scatter(landmark_QC[:,0],landmark_QC[:,1])

In [ ]:
img = cv2.imread(os.path.join(dirpath,'base_20211127204735bcf80943882c47a38340d33f9dbf5140.jpg'))

In [ ]:
plt.imshow(a[...,::-1])

In [ ]:
imm = preprocess(img)

NameError: name 'alive_model' is not defined

In [ ]:
a = plot_lads(landermark_98,frame)

NameError: name 'landermark_98' is not defined

In [ ]:
bounding_boxes = detect_face(imm)

NameError: name 'imm' is not defined

In [ ]:
bounding_boxes

NameError: name 'bounding_boxes' is not defined

In [ ]:
get_eyes_status(landermark_98)

NameError: name 'landermark_98' is not defined

In [ ]:
aligned_sum

NameError: name 'aligned_sum' is not defined